<a href="https://colab.research.google.com/github/cocorini/KT_AIVLE/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_6%EC%B0%A8_EfficientNetB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Augmentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)
* 성능 가이드
    * Accuracy : 0.34~0.74

### 경로 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 라이브러리 불러오기

In [ ]:
import zipfile, os, random, shutil, glob, yaml, json
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
path = '/content/drive/MyDrive/2023.11.01_미니프로젝트 6차_실습자료/Car_Images/'

#### 데이터 조회

In [ ]:
# 폴더별 이미지 데이터 갯수 확인

print(f"정상 차량 이미지 데이터는 {len(glob.glob(path+'normal/*'))}장 입니다.")
print(f"파손 차량 이미지 데이터는 {len(glob.glob(path+'abnormal/*'))}장 입니다.")

정상 차량 이미지 데이터는 302장 입니다.
파손 차량 이미지 데이터는 303장 입니다.


#### Y : 클래스 만들기
- 전체 데이터에 대한 Y를 생성합니다.
- normal, abnormal 데이터의 수를 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
Y = ['0']*302
Y += ['1']*303

#### X : 데이터 리스트 통합
- 전체 이미지 데이터를 하나의 리스트로 통합합니다.

In [ ]:
X = []
X += glob.glob(path+'normal/*')
X += glob.glob(path+'abnormal/*')

#### 데이터셋 분리
- 데이터 스플릿의 비율 -> train set : validation set : test set = 8 : 1: 1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

In [ ]:
len(x_train), len(x_val), len(x_test)

(544, 30, 31)

#### x_train, x_val, x_test 모두 280 * 280 * 3의 형태로 만들어서 리스트로 분류하기 -> 이미지를 로드하는 과정에서 사이즈를 정해야한다.

In [ ]:
category = ['0', '1']

img_h = 280
img_w = 280

X_train=[]
X_val=[]
X_test=[]

for file_name in x_train:
    img = load_img(
        path = file_name,
        color_mode='rgb',
        target_size=(img_h, img_w),
        interpolation='nearest',
        keep_aspect_ratio=True
    )
    X_train.append(img)

for file_name in x_val:
    img = load_img(
        path = file_name,
        color_mode='rgb',
        target_size=(img_h, img_w),
        interpolation='nearest',
        keep_aspect_ratio=True
    )
    X_val.append(img)

for file_name in x_test:
    img = load_img(
        path = file_name,
        color_mode='rgb',
        target_size=(img_h, img_w),
        interpolation='nearest',
        keep_aspect_ratio=True
    )
    X_test.append(img)

In [ ]:
for i in range(len(X_train)):
    arr = img_to_array(X_train[i])
    X_train[i] = arr

for i in range(len(X_val)):
    arr = img_to_array(X_val[i])
    X_val[i] = arr

for i in range(len(X_test)):
    arr = img_to_array(X_test[i])
    X_test[i] = arr

In [ ]:
X_train = np.array(X_train)
X_train = np.expand_dims(X_train, axis=-1)
X_train = np.squeeze(X_train, axis=-1)

X_val = np.array(X_val)
X_val = np.expand_dims(X_val, axis=-1)
X_val = np.squeeze(X_val, axis=-1)

X_test = np.array(X_test)
X_test = np.expand_dims(X_test, axis=-1)
X_test = np.squeeze(X_test, axis=-1)

* 데이터 별 차원 확인하기

In [ ]:
X_train.shape, X_val.shape, X_test.shape

((544, 280, 280, 3), (30, 280, 280, 3), (31, 280, 280, 3))

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [ ]:
y_train.shape, y_val.shape, y_test.shape

((544,), (30,), (31,))

* y를 to_categorical로 인코딩 해주기

In [ ]:
y_tr_category = to_categorical(y_train)
y_val_category = to_categorical(y_val)
y_te_category = to_categorical(y_test)

* early_stopping 정의

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
)

* 딥러닝 라이브러리 불러오기

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D,

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Model

### EfficientNetB1 모델을 backbone으로 transferlearning 진행

In [ ]:
model = tf.keras.applications.efficientnet.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(280, 280, 3),
    pooling=None,
)

In [ ]:
output = model.output

In [ ]:
output

<KerasTensor: shape=(None, 9, 9, 1280) dtype=float32 (created by layer 'top_activation')>

* 내가 원하는 모델은 2진분류 모델이기 때문에, 1000개 분류 모델을 2개짜리로 바꿔준다.

In [ ]:
x = GlobalMaxPooling2D()(output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)

x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)

output = Dense(2, activation='sigmoid', name='output')(x)

model = Model(inputs = model.input, outputs = output)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 280, 280, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 280, 280, 3)          0         ['input_8[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 280, 280, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 280, 280, 3)          0         ['normalization_1[0][0]'

* 학습하기

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_tr_category, epochs = 10,
          validation_data=(X_val, y_val_category),
          callbacks=[early_stopping]
          )

Epoch 1/10
17/17 [==============================] - 73s 855ms/step - loss: 0.4736 - accuracy: 0.8088 - val_loss: 1.1897 - val_accuracy: 0.5333
Epoch 2/10
17/17 [==============================] - 10s 597ms/step - loss: 0.1918 - accuracy: 0.9596 - val_loss: 0.0606 - val_accuracy: 0.9667
Epoch 3/10
17/17 [==============================] - 10s 595ms/step - loss: 0.1070 - accuracy: 0.9816 - val_loss: 0.0519 - val_accuracy: 0.9667
Epoch 4/10
17/17 [==============================] - 10s 599ms/step - loss: 0.0896 - accuracy: 0.9835 - val_loss: 0.0383 - val_accuracy: 0.9667
Epoch 5/10
17/17 [==============================] - 10s 605ms/step - loss: 0.0638 - accuracy: 0.9835 - val_loss: 0.0788 - val_accuracy: 0.9667
Epoch 6/10
17/17 [==============================] - 10s 603ms/step - loss: 0.0616 - accuracy: 0.9853 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 7/10
17/17 [==============================] - 10s 599ms/step - loss: 0.0800 - accuracy: 0.9779 - val_loss: 0.1297 - val_accuracy: 0.9667

* 성능 평가: classification_report 이용

In [ ]:
y_pred = model.predict(X_test)

y_pred_class=[]
for pred in y_pred:
    if pred[0] < pred[1]:
        y_pred_class.append('1')
    else:
        y_pred_class.append('0')

print(classification_report(y_test, y_pred_class))

1/1 [==============================] - 3s 3s/step
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        16
           1       1.00      0.80      0.89        15

    accuracy                           0.90        31
   macro avg       0.92      0.90      0.90        31
weighted avg       0.92      0.90      0.90        31

